# Validation script for Imagenet models

## Overview
Use this notebook to verify the accuracy of a trained model on a validation set. It support all models {really? how about we list what is supported below?} trained on Imagenet {CamelCase!} dataset ([ILSVRC2012](http://www.image-net.org/challenges/LSVRC/2012/) and onwards).

## Model Support in This Demo

{list with links}

## Prerequisites
{list them (add min versions)}
Also, you should have already trained imagenet and have the data somewhere...

What is gluoncv!?

## Guidelines for usage
* The .onnx file for the model must be in the root folder {in / ? - that seems weird}
* Install dependencies using cell below
* Imagenet dataset must be downloaded and extracted in the required directory structure. Refer to the guidelines in the dataset section of [squeezenet readme](squeezenet/README.md).
{the above sends you to an new page, which then doesn't say much and sends you to imagenet_prep.md. Add clarity and make use of the #anchors to go to the appropriate page sections directly.}

In [ ]:
# Install dependencies
!pip install mxnet-cu90mkl #tested on this version, can use other versions
!pip install gluoncv
!pip install numpy
!pip install matplotlib

In [ ]:
# Import required packages
import matplotlib
matplotlib.use('Agg')

import mxnet as mx
import numpy as np
from mxnet import gluon, nd
from mxnet.gluon.data.vision import transforms

from gluoncv.data import imagenet
from collections import namedtuple

In [ ]:
# choose context as cpu or gpu(recommended)
# ctx = [mx.cpu()]
ctx = [mx.gpu(0)]

# path to imagenet dataset folder
data_dir = '../imagenet/img_dataset'

# batch size (set to 1 for cpu)
batch_size = 128

# number of preprocessing workers
num_workers = 32

# model name
model_name = 'vgg16'

In [ ]:
# Import from ONNX format to MXNet symbols and params
sym, arg_params, aux_params = import_model(model_name+'.onnx')

In [ ]:
# Incase ONNX file not available - load MXNet symbols and params directly, skip otherwise
sym, arg_params, aux_params = mx.model.load_checkpoint(model_name, 0)

Add clarity on each step... what is it doing?

In [ ]:
# Define model
mod = mx.mod.Module(symbol=sym, context=ctx, label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

Since you're reusing a lot of the same code about resizing and normalizing, pull the values out, make them variables in a setup section and discuss them. Then you get to reuse that in each notebook.

Explain what is happening in the next section.

In [ ]:
# Define evaluation metrics
acc_top1 = mx.metric.Accuracy()
acc_top5 = mx.metric.TopKAccuracy(5)

# Define image transforms
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

# Prepare input
val_data = gluon.data.DataLoader(
    imagenet.classification.ImageNet(data_dir, train=False).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)

Capital Batch?
Explain what this section is doing.

In [ ]:
# Compute evaluations
Batch = namedtuple('Batch', ['data'])
acc_top1.reset()
acc_top5.reset()
num_batches = int(50000/batch_size)
print('[0 / %d] batches done'%(num_batches))
for i, batch in enumerate(val_data):
    data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
    label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
    mod.forward(Batch([data[0]]))
    outputs=mod.get_outputs()
    acc_top1.update(label, outputs)
    acc_top5.update(label, outputs)
    if (i+1)%50==0:
        print('[%d / %d] batches done'%(i+1,num_batches))

In [ ]:
# Print results
_, top1 = acc_top1.get()
_, top5 = acc_top5.get()
print('top1 error:',1-top1,'; top-5 error:',1-top5)

{This validation is backwards isn't? I'm used to seeing something accuracy, not error.}